## Name :-  Amar Babuta

## Student Id:- 1031452

##### The commented part of the code is for expermental purpose

#### Importing Required libraries

In [1]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.ensemble import VotingClassifier
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import nltk as nlp
import string
import re
import os
np.random.seed(42)

#### Loading Train dataset

In [2]:
#Read the data
df=pd.read_json('train.json')
#Get shape and head
print(df.shape)
df.head()

(2, 1168)


,train-0,train-1,train-2,train-3,train-4,train-5,train-6,train-7,train-8,train-9,...,train-1158,train-1159,train-1160,train-1161,train-1162,train-1163,train-1164,train-1165,train-1166,train-1167
text,why houston flooding isn‘t a sign of climate c...,The U.N. Intergovernmental Panel on Climate Ch...,Bureau Now Sets Strict Limits on Cooling\nOVER...,The Dirty Extractive Underbelly of Clean Energ...,why climate change seems to have faded from th...,ipcc achieves zero credibility - our coalition...,Riots in Chile forced Chilean President Sebast...,claims of climate calamities made 100 months a...,Bureau’s Report Beyond Belief\nThe release of ...,the climate change emergency is a pagan cult\n...,...,Led by Colorado Attorney General (AG) Cynthia ...,earth day 2019 - not one single environmental ...,what navigator captain james cook might have s...,an avalanche of global warming alarmism is abo...,the world isn't going to halve co2 emissions b...,"From Alberta to Australia, from Finland to Fra...",Lake Tahoe is enjoying a third consecutive yea...,New climate tests for pipelines are unnecessar...,Al Gore Claims Wind and Solar are Now Cheaper ...,"This paper from Bruce Colbert, executive direc..."
label,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [3]:
#Transpose the dataframe
df1=df.T
df1

,text,label
train-0,why houston flooding isn‘t a sign of climate c...,1
train-1,The U.N. Intergovernmental Panel on Climate Ch...,1
train-2,Bureau Now Sets Strict Limits on Cooling\nOVER...,1
train-3,The Dirty Extractive Underbelly of Clean Energ...,1
train-4,why climate change seems to have faded from th...,1
...,...,...
train-1163,"From Alberta to Australia, from Finland to Fra...",1
train-1164,Lake Tahoe is enjoying a third consecutive yea...,1
train-1165,New climate tests for pipelines are unnecessar...,1
train-1166,Al Gore Claims Wind and Solar are Now Cheaper ...,1


#### Loading the External dataset

In [4]:
#Read the data
df2=pd.read_json('external_data.json')
#Get shape and head
print(df2.shape)
df2.head()

(4168, 2)


,text,label
0,ISLAMABAD: Finance Minister Ishaq Dar presente...,0
1,ISLAMABAD: As Senate Chairman Raza Rabbani rul...,0
2,NEW DELHI: India unveiled a fire-fighting budg...,0
3,strong>ISLAMABAD: Prime Minister Nawaz Sharif ...,0
4,strong>LONDON/NEW DELHI/MOSCOW: A multi-billio...,0


In [5]:
#concating the train and external data in new data frame
df4=pd.concat([df1,df2])

In [6]:
#converting to orient records
df4.to_json('newtrain.json', orient='records')
newtrain = pd.read_json('newtrain.json')
newtrain

,text,label
0,why houston flooding isn‘t a sign of climate c...,1
1,The U.N. Intergovernmental Panel on Climate Ch...,1
2,Bureau Now Sets Strict Limits on Cooling\nOVER...,1
3,The Dirty Extractive Underbelly of Clean Energ...,1
4,why climate change seems to have faded from th...,1
...,...,...
5331,veteran martinez wins thai title conchita mart...,0
5332,everton s weir cools euro hopes everton defend...,0
5333,tottenham bid &#163;8m for forest duo nottingh...,0
5334,solskjaer raises hopes of return manchester un...,0


In [7]:
#Get the labels from newtrain
labels=newtrain.label
labels.head()
labels=labels.astype('int')

In [8]:
zero = newtrain['label'] == 0
zero2 = newtrain[zero]
print(zero2)

                                                   text  label
1168  ISLAMABAD: Finance Minister Ishaq Dar presente...      0
1169  ISLAMABAD: As Senate Chairman Raza Rabbani rul...      0
1170  NEW DELHI: India unveiled a fire-fighting budg...      0
1171  strong>ISLAMABAD: Prime Minister Nawaz Sharif ...      0
1172  strong>LONDON/NEW DELHI/MOSCOW: A multi-billio...      0
...                                                 ...    ...
5331  veteran martinez wins thai title conchita mart...      0
5332  everton s weir cools euro hopes everton defend...      0
5333  tottenham bid &#163;8m for forest duo nottingh...      0
5334  solskjaer raises hopes of return manchester un...      0
5335  blunkett hints at election call ex-home secret...      0

[4168 rows x 2 columns]


#### Loading the dev dataset

In [9]:
dev_df=pd.read_json('dev.json')
#making transpose of dev_df
dev_df1=dev_df.T
dev_df1

,text,label
dev-0,Are Climate Models Overpredicting Global Warmi...,1
dev-1,"The latest National Climate Assessment, releas...",1
dev-2,Climate Strike Kids Cool on Real Action\nA pop...,1
dev-3,Morrison a ‘predatory’ centrist on climate pol...,0
dev-4,CNN’s Town Hall Full of Hot Air\nSixty-three s...,1
...,...,...
dev-95,Texas Man Guilty of Kidnapping Still-Missing W...,0
dev-96,Climate campaigners condemn 'insidious' cockta...,0
dev-97,Forget the climate emergency: they need a Pied...,1
dev-98,Why don’t we treat the climate crisis with the...,0


In [10]:
#converting to orient records
dev_df1.to_json('dev-1.json', orient='records')
dev1 = pd.read_json('dev-1.json')
dev1

,text,label
0,Are Climate Models Overpredicting Global Warmi...,1
1,"The latest National Climate Assessment, releas...",1
2,Climate Strike Kids Cool on Real Action\nA pop...,1
3,Morrison a ‘predatory’ centrist on climate pol...,0
4,CNN’s Town Hall Full of Hot Air\nSixty-three s...,1
...,...,...
95,Texas Man Guilty of Kidnapping Still-Missing W...,0
96,Climate campaigners condemn 'insidious' cockta...,0
97,Forget the climate emergency: they need a Pied...,1
98,Why don’t we treat the climate crisis with the...,0


In [11]:
#Get the labels from dev dataset
dev_labels=dev_df1.label
dev_labels.head()
dev_labels=dev_labels.astype('int')

#### Loading test dataset

In [12]:
test_df=pd.read_json('test-unlabelled.json')
#making transpose of test_df
test_df1=test_df.T
test_df1

,text
test-0,"Marketing Executive Somehow Unable To Stop, Pr..."
test-1,WASHINGTON Republican presidential front-runn...
test-2,Turkey Could Be Taking A Big Step Backwards In...
test-3,New York Keep your kids away from these Santa...
test-4,Global warming is causing more frequent heatwa...
...,...
test-1405,Warwick asks voters to back radical council ta...
test-1406,"Pele, Diego Maradona and Franz Beckenbaeur wil..."
test-1407,Electric cars produce less CO2 than petrol veh...
test-1408,Latest Climate Report Feeds into Alarmist Fear...


In [13]:
#converting to orient records
test_df1.to_json('test-unlabelled1.json', orient='records')
test1 = pd.read_json('test-unlabelled1.json')
test1

,text
0,"Marketing Executive Somehow Unable To Stop, Pr..."
1,WASHINGTON Republican presidential front-runn...
2,Turkey Could Be Taking A Big Step Backwards In...
3,New York Keep your kids away from these Santa...
4,Global warming is causing more frequent heatwa...
...,...
1405,Warwick asks voters to back radical council ta...
1406,"Pele, Diego Maradona and Franz Beckenbaeur wil..."
1407,Electric cars produce less CO2 than petrol veh...
1408,Latest Climate Report Feeds into Alarmist Fear...


#### Different Feature selection Approach and it's execution

######  Approach A) Using normal features

##### Creating Normal features for newtrain dataset

In [ ]:
# Tokenizing newtrain text

In [ ]:
# for word in newtrain['text']:
#     sentences=nltk.sent_tokenize(word)
#     print(len(sentences))

In [ ]:
# Removal of Punctuation from text

In [ ]:
# def remove_punct(text):
#     text  = "".join([char for char in text if char not in string.punctuation])
#     text = re.sub('[0-9]+', '', text)
#     return text

In [ ]:
# Initialize Wordnet Lemmatizer

In [ ]:
# lemmatizer = WordNetLemmatizer()

In [ ]:
# Removal of stopwords from the text by converting to lowercase and lemmatizing the text

In [ ]:
# new_str = []
# str1 =" "
# for word in newtrain['text']:
#     sentences=nltk.sent_tokenize(word)
#     for item in range(len(sentences)):
#         words = nltk.word_tokenize(remove_punct(sentences[item]).lower())
#         words = [lemmatizer.lemmatize(word) for word in words if word not in set(stopwords.words('english'))]
#         sentences[item] = ' '.join(words)
#     string_1 = str1.join(sentences)
#     new_str.append(string_1)

In [ ]:
# new = pd.DataFrame(new_str)
# newtrain['preprocessed_text']= new
# newtrain = newtrain[['text','preprocessed_text','label']]
# newtrain

##### Creating Normal features for dev dataset

In [ ]:
# Tokenizing dev dataset

In [ ]:
# for word in dev1['text']:
#     sentences=nltk.sent_tokenize(word)
#     print(len(sentences))

In [ ]:
# Removal of stopwords, punctuation from the text by converting to lowercase and lemmatizing the text

In [ ]:
# new_str1 = []
# str2 =" "
# for word in dev1['text']:
#     sentences1=nltk.sent_tokenize(word)
#     for item in range(len(sentences1)):
#         words = nltk.word_tokenize(remove_punct(sentences1[item]).lower())
#         words = [lemmatizer.lemmatize(word) for word in words if word not in set(stopwords.words('english'))]
#         sentences1[item] = ' '.join(words)
#     string_2 = str1.join(sentences1)
#     new_str1.append(string_2)

In [ ]:
# dev = pd.DataFrame(new_str1)
# dev1['preprocessed_text']= dev
# dev1= dev1[['text','preprocessed_text','label']]
# dev1

##### Creating Normal features for test dataset

In [ ]:
# Tokenizing test dataset

In [ ]:
# for word in test1['text']:
#     sentences=nltk.sent_tokenize(word)
#     print(len(sentences))

In [ ]:
# Removal of stopwords, punctuation from the text by converting to lowercase and lemmatizing the text

In [ ]:
# new_str2 = []
# str3 =" "
# for word in test1['text']:
#     sentences2=nltk.sent_tokenize(word)
#     for item in range(len(sentences2)):
#         words = nltk.word_tokenize(remove_punct(sentences2[item]).lower())
#         words = [lemmatizer.lemmatize(word) for word in words if word not in set(stopwords.words('english'))]
#         sentences2[item] = ' '.join(words)
#     string_3 = str3.join(sentences2)
#     new_str2.append(string_3)

In [ ]:
# test = pd.DataFrame(new_str2)
# test1['preprocessed_text']= test
# test1= test1[['text','preprocessed_text']]
# test1

###### Applying normal features to train model

In [ ]:
# #Split the dataset
# x_train,x_test,y_train,y_test=train_test_split(newtrain['preprocessed_text'], labels, test_size=0.3, random_state=7)
# y_train=y_train.astype('int')
# y_test=y_test.astype('int')

In [ ]:
# bnb = BernoulliNB()
# bnb.fit(x_train, y_train)

In [ ]:
# bnb_predict = bnb.predict(x_test)
# print("accuracy: {}%".format(round(accuracy_score(y_test, bnb_predict)*100,2)))

##### Approach b) YAKE(Yet Another Keyword Extraction) for Bag of words

In [ ]:
# #Manipulating newtrain dataset to apply YAKE 

# newtrain.insert(0, 'ID', range(0, 0 + len(newtrain)))
# newtrain

In [ ]:
# #Applying YAKE on newtrain dataset


# import yake

# final={}
# for i in range(len(newtrain['text'])):
#     trial = []
#     id = newtrain['ID'][i]
#     text = newtrain['text'][i]
#     # assuming default parameters
#     simple_kwextractor = yake.KeywordExtractor()
#     keywords = simple_kwextractor.extract_keywords(text)
    
#     # for kw in keywords:
#     # 	print(kw)
    
#     # specifying parameters
#     max_ngram_size = 3
#     custom_kwextractor = yake.KeywordExtractor(lan="en", n = max_ngram_size, dedupLim=0.9, dedupFunc='seqm', windowsSize=1, top=20, features=None)
    
#     keywords = custom_kwextractor.extract_keywords(text)
    
#     for (a,b) in keywords:
#         if b<0.6:
#             #print (a)
#             trial.append(a)
            
#     if id not in final:
#         final[id]=[]
#     final[id].append(trial)

# print(final)

In [ ]:
# train_frame = pd.DataFrame(final)
# train_frame= train_frame.T
# newtrain['keyword'] = train_frame
# newtrain = newtrain[['ID','text','keyword' , 'label']]
# newtrain

In [ ]:
# def list_to_str(req):
#     return ' '.join(req)

In [ ]:
# newtrain['keyword'] =newtrain['keyword'].apply(list_to_str)
#newtrain

In [ ]:
# def unique(text):
#     for word in text:
#         text = text.split()
#         text = set(text)
#         return text

In [ ]:
# newtrain['title_unq'] = newtrain['keyword'].apply(lambda x: unique(x))

In [ ]:
# def set_to_lst(req):
#     return list(req)

In [ ]:
# newtrain['title_unq'] = newtrain['title_unq'].apply(set_to_lst)

In [ ]:
# newtrain['title_unq'] =newtrain['title_unq'].apply(list_to_str)

In [ ]:
# newtrain=newtrain[['ID','text','keyword','title_unq','label']]
#newtrain

In [ ]:
# #manipulating Dev Dataset
# dev1.insert(0, 'ID', range(0, 0 + len(dev1)))
# dev1

In [ ]:
# #Applying YAKE on Dev Dataset

# import yake

# final={}
# for i in range(len(dev1['text'])):
#     trial = []
#     id = dev1['ID'][i]
#     text = dev1['text'][i]
#     # assuming default parameters
#     simple_kwextractor = yake.KeywordExtractor()
#     keywords = simple_kwextractor.extract_keywords(text)
    
#     # for kw in keywords:
#     # 	print(kw)
    
#     # specifying parameters
#     max_ngram_size = 3
#     custom_kwextractor = yake.KeywordExtractor(lan="en", n = max_ngram_size, dedupLim=0.9, dedupFunc='seqm', windowsSize=1, top=20, features=None)
    
#     keywords = custom_kwextractor.extract_keywords(text)
    
#     for (a,b) in keywords:
#         if b<0.6:
#             #print (a)
#             trial.append(a)
            
#     if id not in final:
#         final[id]=[]
#     final[id].append(trial)

# print(final)

In [ ]:
# dic_frame = pd.DataFrame(final)
# dic_frame= dic_frame.T
# dev1['keyword'] = dic_frame
# dev1 = dev1[['ID','text','keyword' , 'label']]
# dev1

In [ ]:
# dev1['keyword'] =dev1['keyword'].apply(list_to_str)
# dev1

In [ ]:
# dev1['title_unq'] = dev1['keyword'].apply(lambda x: unique(x))

In [ ]:
# dev1['title_unq'] = dev1['title_unq'].apply(set_to_lst)

In [ ]:
# dev1['title_unq'] =dev1['title_unq'].apply(list_to_str)

In [ ]:
# dev1=dev1[['ID','text','keyword','title_unq','label']]
# dev1

In [ ]:
# #Manipulating Test Dataset
# test1.insert(0, 'ID', range(0, 0 + len(test1)))
# test1

In [ ]:
# #Applying YAKE on test dataset

# import yake

# final={}
# for i in range(len(newtrain['text'])):
#     trial = []
#     id = newtrain['ID'][i]
#     text = newtrain['text'][i]
#     # assuming default parameters
#     simple_kwextractor = yake.KeywordExtractor()
#     keywords = simple_kwextractor.extract_keywords(text)
    
#     # for kw in keywords:
#     # 	print(kw)
    
#     # specifying parameters
#     max_ngram_size = 3
#     custom_kwextractor = yake.KeywordExtractor(lan="en", n = max_ngram_size, dedupLim=0.9, dedupFunc='seqm', windowsSize=1, top=20, features=None)
    
#     keywords = custom_kwextractor.extract_keywords(text)
    
#     for (a,b) in keywords:
#         if b<0.6:
#             #print (a)
#             trial.append(a)
            
#     if id not in final:
#         final[id]=[]
#     final[id].append(trial)

# print(final)

In [ ]:
# test_frame = pd.DataFrame(final)
# test_frame= test_frame.T
# test1['keyword'] = test_frame
# test1

In [ ]:
# test1['keyword'] = test1['keyword'].apply(list_to_str)
# test1

In [ ]:
# test1['title_unq'] = test1['keyword'].apply(lambda x: unique(x))
# test1['title_unq'] = test1['title_unq'].apply(set_to_lst)
# test1['title_unq'] =test1['title_unq'].apply(list_to_str)
# test1

###### Applying YAKE with TF-IDF for Feature Extraction

In [ ]:
# #Initialize a TfidfVectorizer
# tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.7)
# #Fit and transform train set
# tfidf_train=tfidf_vectorizer.fit_transform(newtrain['title_unq'])
# tfidf_train

##### Applying extracted feature to train model

In [ ]:
# #Split the dataset
# x_train,x_test,y_train,y_test=train_test_split(tfidf_train, labels, test_size=0.3, random_state=7)
# y_train=y_train.astype('int')
# y_test=y_test.astype('int')

In [ ]:
# linear_svm=LinearSVC()
# linear_svm.fit(x_train,y_train)

In [ ]:
# lin_svm_predict = linear_svm.predict(x_test)
# print("accuracy: {}%".format(round(accuracy_score(y_test, lin_svm_predict)*100,2)))

###### Approach c) TF-IDF Vectorizer

In [14]:
#Initialize a TfidfVectorizer
tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.7)
#Fit and transform train set
tfidf_train=tfidf_vectorizer.fit_transform(newtrain['text'])
tfidf_train

<5336x47556 sparse matrix of type '<class 'numpy.float64'>'
	with 793776 stored elements in Compressed Sparse Row format>

In [15]:
#Split the dataset
x_train,x_test,y_train,y_test=train_test_split(tfidf_train, labels, test_size=0.3, random_state=7)
y_train=y_train.astype('int')
y_test=y_test.astype('int')

##### Testing on Multinomial Naive Bayes

In [ ]:
# mnb = MultinomialNB()
# mnb.fit(x_train, y_train)

In [ ]:
# mnb_predict = mnb.predict(x_test)
# print("accuracy: {}%".format(round(accuracy_score(y_test, mnb_predict)*100,2)))

##### Testing on Support Vector Classifier

In [ ]:
# svc = SVC()
# svc.fit(x_train, y_train)

In [ ]:
# svc_predict = svc.predict(x_test)
# print("accuracy: {}%".format(round(accuracy_score(y_test, svc_predict)*100,2)))

##### Hyperparameter tuning of SVC

In [ ]:
# #Initialize a TfidfVectorizer
# tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.7)
# #Fit and transform train set
# tfidf_train=tfidf_vectorizer.fit_transform(dev1['text'])
# tfidf_train

In [ ]:
# #Split the dataset
# x_train,x_test,y_train,y_test=train_test_split(tfidf_train, dev1.label, test_size=0.3, random_state=7)
# y_train=y_train.astype('int')
# y_test=y_test.astype('int')

In [ ]:
# param_grid = {'C': [0.1,1, 10, 100 , 1000], 
#                 'gamma': [1,0.1,0.01,0.001 , 0.0001],
#                 'kernel': ['rbf', 'poly', 'sigmoid' , 'linear']}

In [ ]:
# grid = GridSearchCV(svm,param_grid,refit=True,verbose=2)
# grid.fit(x_train,y_train)

In [ ]:
# print(grid.best_estimator_)

##### Applying Optimized Parameter on newtrain

In [ ]:
# #Initialize a TfidfVectorizer
# tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.7)
# #Fit and transform train set
# tfidf_train=tfidf_vectorizer.fit_transform(newtrain['text'])
# tfidf_train

In [ ]:
# #Split the dataset
# x_train,x_test,y_train,y_test=train_test_split(tfidf_train, labels, test_size=0.3, random_state=7)
# y_train=y_train.astype('int')
# y_test=y_test.astype('int')

In [ ]:
# svc = SVC(C=10, gamma=1, kernel='sigmoid')
# svc.fit(x_train, y_train)

In [ ]:
# svc_predict = svc.predict(x_test)
# print("accuracy: {}%".format(round(accuracy_score(y_test, svc_predict)*100,2)))

##### Approach d) Training the model with ensemble approach

In [ ]:
# #Initialize a TfidfVectorizer
# tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.7)
# #Fit and transform train set
# tfidf_train=tfidf_vectorizer.fit_transform(newtrain['text'])
# tfidf_train

In [ ]:
# #Split the dataset
# x_train,x_test,y_train,y_test=train_test_split(tfidf_train, labels, test_size=0.3, random_state=7)
# y_train=y_train.astype('int')
# y_test=y_test.astype('int')

In [ ]:
# ensemble=VotingClassifier(estimators=[('LR', lr), ('SVC', svc)], 
#                        voting='hard', weights=[1,2]).fit(x_train,y_train)
# print('The accuracy for Logistic Regression and Support Vector Machines is:',ensemble.score(x_test,y_test))

In [ ]:
# ensemble_predict = ensemble.predict(x_test)
# print("accuracy: {}%".format(round(accuracy_score(y_test, ensemble_predict)*100,2)))

##### Final model training

In [16]:
#Initialize a TfidfVectorizer
tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.7)
#Fit and transform train set
tfidf_train=tfidf_vectorizer.fit_transform(newtrain['text'])
tfidf_train

<5336x47556 sparse matrix of type '<class 'numpy.float64'>'
	with 793776 stored elements in Compressed Sparse Row format>

In [17]:
#Split the dataset
x_train,x_test,y_train,y_test=train_test_split(tfidf_train, labels, test_size=0.3, random_state=7)
y_train=y_train.astype('int')
y_test=y_test.astype('int')

In [18]:
lr = LogisticRegression()
lr.fit(x_train, y_train)

C:\Users\AMAR\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [19]:
lr_predict = lr.predict(x_test)
print("accuracy: {}%".format(round(accuracy_score(y_test, lr_predict)*100,2)))

accuracy: 97.31%


##### Hyperparameter tuning of Logistic Regression

In [20]:
#Initialize a TfidfVectorizer
tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.7)
#Fit and transform train set
tfidf_train=tfidf_vectorizer.fit_transform(dev1['text'])
tfidf_train

<100x9606 sparse matrix of type '<class 'numpy.float64'>'
	with 26064 stored elements in Compressed Sparse Row format>

In [21]:
#Split the dataset
x_train,x_test,y_train,y_test=train_test_split(tfidf_train, dev1.label, test_size=0.3, random_state=7)
y_train=y_train.astype('int')
y_test=y_test.astype('int')

In [22]:
lr = LogisticRegression()
lr.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [23]:
lr_predict = lr.predict(x_test)
print("accuracy: {}%".format(round(accuracy_score(y_test, lr_predict)*100,2)))

accuracy: 86.67%


In [28]:
penalty = ['l1', 'l2']
C = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
class_weight = [{1:0.5, 0:0.5}, {1:0.4, 0:0.6}, {1:0.6, 0:0.4}, {1:0.7, 0:0.3}]
solver = ['liblinear', 'saga']

param_grid = dict(penalty=penalty,
                  C=C,
                  class_weight=class_weight,
                  solver=solver)

grid = GridSearchCV(estimator=lr,
                    param_grid=param_grid,
                    scoring='roc_auc',
                    verbose=1,
                    n_jobs=-1)
grid_result = grid.fit(x_train, y_train)

print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

C:\Users\AMAR\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 128 candidates, totalling 384 fits
Best Score:  0.9519480519480518
Best Params:  {'C': 100, 'class_weight': {1: 0.7, 0: 0.3}, 'penalty': 'l1', 'solver': 'saga'}


[Parallel(n_jobs=-1)]: Done 384 out of 384 | elapsed:    1.7s finished
C:\Users\AMAR\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\AMAR\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [29]:
lr = LogisticRegression(C=1000, class_weight={1: 0.7, 0: 0.3}, penalty='l1',solver='saga')
lr.fit(x_train, y_train)

LogisticRegression(C=1000, class_weight={1: 0.7, 0: 0.3}, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l1', random_state=None,
          solver='saga', tol=0.0001, verbose=0, warm_start=False)

In [30]:
lr_predict = lr.predict(x_test)
print("accuracy: {}%".format(round(accuracy_score(y_test, lr_predict)*100,2)))

accuracy: 83.33%


In [31]:
score=accuracy_score(y_test,lr_predict)
print(f'Accuracy: {round(score*100,2)}%')

cm=confusion_matrix(y_test,lr_predict, labels=[1,0])
print ("\n Confusion Matrix : \n")
print (cm)

p, r, f, _ = precision_recall_fscore_support(y_test,lr_predict, pos_label=1, average="binary");

print("\nPerformance on the positive class (documents with misinformation):")
print("Precision =", p)
print("Recall    =", r)
print("F1        =", f)

Accuracy: 83.33%

 Confusion Matrix : 

[[13  2]
 [ 3 12]]

Performance on the positive class (documents with misinformation):
Precision = 0.8125
Recall    = 0.8666666666666667
F1        = 0.8387096774193549


##### Applying Optimized Parameter on newtrain

In [32]:
#Initialize a TfidfVectorizer
tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.7)
#Fit and transform train set
tfidf_train=tfidf_vectorizer.fit_transform(newtrain['text'])
tfidf_train

<5336x47556 sparse matrix of type '<class 'numpy.float64'>'
	with 793776 stored elements in Compressed Sparse Row format>

In [33]:
#Split the dataset
x_train,x_test,y_train,y_test=train_test_split(tfidf_train, labels, test_size=0.3, random_state=7)
y_train=y_train.astype('int')
y_test=y_test.astype('int')

In [34]:
lr = LogisticRegression(C=1000, class_weight={1: 0.7, 0: 0.3}, penalty='l1',solver='saga')
lr.fit(x_train, y_train)

LogisticRegression(C=1000, class_weight={1: 0.7, 0: 0.3}, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l1', random_state=None,
          solver='saga', tol=0.0001, verbose=0, warm_start=False)

In [35]:
lr_predict = lr.predict(x_test)
print("accuracy: {}%".format(round(accuracy_score(y_test, lr_predict)*100,2)))

accuracy: 99.44%


##### Predicting the labels of test data

In [36]:
x_train=tfidf_train
y_train=labels

In [37]:
test_pred=test1['text']

In [38]:
tfidf_pred=tfidf_vectorizer.transform(test_pred)

In [39]:
lr_pred=lr.predict(tfidf_pred)

In [40]:
test_df1['label']= lr_pred.tolist()

In [41]:
df_TestOutput=test_df1[['label']].copy(deep=True)

In [42]:
df_TestOutput=df_TestOutput.T

In [43]:
df_TestOutput.to_json ('test-output.json')